In [1]:
%load_ext autoreload
%autoreload
%autoreload 2

In [2]:
from synthetic_torch_helpers import SynH5Dataset, ConvModSyn, H5Dataset

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss, Metric, Accuracy
from ignite.handlers import ModelCheckpoint, EarlyStopping

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score

import torch
import torchvision
import datetime

import numpy as np
import matplotlib.pyplot as plt

***

# Setting up Data

#### Reading in Eboss Data

In [3]:
eboss = H5Dataset("../astrogravlensing/Data/eboss_flux_full+.hdf5", keys=['res_flux_values', 'flux_labels'], load_to_memory=True)

In [4]:
eboss.res_flux_values = eboss.res_flux_values[:, np.newaxis, :]

In [5]:
eboss.res_flux_values.shape

(281, 1, 4639)

#### Centering the datasets

In [6]:
eboss.res_flux_values -= np.mean(eboss.res_flux_values, axis=0)

### Scaling the datasets

In [7]:
flux_mss = MinMaxScaler()
eboss.res_flux_values = flux_mss.fit_transform(eboss.res_flux_values[:, 0, :]).reshape(-1, 1, 4639)

### Data Loaders

In [8]:
train_loader = torch.utils.data.DataLoader(eboss, batch_size=8, shuffle=True)

***

# Constructing the Model

In [9]:
conv_config = [
    (1, 64, 10),
    (64, 32, 10),
# Max Pooling
    (32, 12, 2),
    (12, 6, 5),
]
pooling_ixs = {1 : 2, 3: 2}


full_config = [
    (6912, 1024),
    # Dropout
    (1024, 512),
    (512, 1)
]
dropout_ixs = {0: .5, 1 : .5}

mod = ConvModSyn(conv_config, full_config, pooling_ixs, dropout_ixs, torch.nn.Sigmoid())
mod

ConvModSyn(
  (conv_layers): Sequential(
    (conv_0): Sequential(
      (0): Conv1d(1, 64, kernel_size=(10,), stride=(1,))
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv_1): Sequential(
      (0): Conv1d(64, 32, kernel_size=(10,), stride=(1,))
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (pool_1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv_2): Sequential(
      (0): Conv1d(32, 12, kernel_size=(2,), stride=(1,))
      (1): BatchNorm1d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (conv_3): Sequential(
      (0): Conv1d(12, 6, kernel_size=(5,), stride=(1,))
      (1): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (pool_3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mo

### Optimizer

In [10]:
opt = torch.optim.Adam(mod.parameters(), lr=0.0001)

### Loss Function

In [11]:
loss = torch.nn.BCELoss()

### Setting up gpu device

In [12]:
%%bash
echo $CUDA_VISIBLE_DEVICES

0


In [13]:
print(torch.cuda.is_available())
device = torch.device("cuda")

True


***

# Training Logic

### Setting up trainer and evaluator

In [21]:
def thresholded_output_transform(output):
    y_pred, y = output[0], output[1]
    y_pred = torch.round(y_pred)
    return y_pred, y

In [22]:
trainer = create_supervised_trainer(mod, opt, loss, device=device)
evaluator = create_supervised_evaluator(mod, metrics={'bce': Loss(loss), 'acc': Accuracy(output_transform=thresholded_output_transform)}, device=device)

In [23]:
model_saver = ModelCheckpoint("./models_lensing/sess_{}/".format(datetime.datetime.now()), "reg", create_dir=True, 
                              score_function=lambda eng: eng.state.train_loss, score_name='t_loss', n_saved=5)
early_stopper = EarlyStopping(20, score_function=lambda eng: eng.state.train_loss, trainer=trainer)

### Training Events (Experimenting with ignite for pytorch)

In [24]:
iter_level = 3

In [25]:
@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(trainer):
    if (trainer.state.iteration-1) % iter_level == 0:
        print("Epoch[{}], Iter: {}, Loss: {:.5f}".format(trainer.state.epoch, trainer.state.iteration, trainer.state.output))

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(trainer):
    evaluator.run(train_loader)
    metrics = evaluator.state.metrics
    trainer.state.train_loss = metrics['bce']
    trainer.state.train_acc = metrics['acc']
    print("Training Results - Epoch: {}, Accuracy: {:.5f}, Avg loss: {:.5f}".format(trainer.state.epoch, metrics['acc'], metrics['bce']))

# @trainer.on(Events.EPOCH_COMPLETED)
# def log_validation_results(trainer):
#     evaluator.run(val_loader)
#     metrics = evaluator.state.metrics
#     trainer.state.val_loss = metrics['bce']
#     print("Validation Results - Epoch: {}, SciStandard: {:.5f}, Avg loss: {:.5f}".format(trainer.state.epoch, metrics['sci'], metrics['mse']))

In [26]:
trainer.add_event_handler(Events.EPOCH_COMPLETED, model_saver, {'mod': mod})
# trainer.add_event_handler(Events.EPOCH_COMPLETED, early_stopper)

## Running the training loop

In [27]:
trainer.run(train_loader, max_epochs=100)

Epoch[1], Iter: 1, Loss: 0.19009
Epoch[1], Iter: 4, Loss: 0.65826
Epoch[1], Iter: 7, Loss: 0.08786
Epoch[1], Iter: 10, Loss: 0.04918
Epoch[1], Iter: 13, Loss: 0.11345
Epoch[1], Iter: 16, Loss: 0.24549
Epoch[1], Iter: 19, Loss: 0.43425
Epoch[1], Iter: 22, Loss: 0.15402
Epoch[1], Iter: 25, Loss: 0.29196
Epoch[1], Iter: 28, Loss: 0.10831
Epoch[1], Iter: 31, Loss: 0.18792
Epoch[1], Iter: 34, Loss: 0.13253
Training Results - Epoch: 1, Accuracy: 0.89680, Avg loss: 0.16054
Epoch[2], Iter: 37, Loss: 0.24589
Epoch[2], Iter: 40, Loss: 0.24803
Epoch[2], Iter: 43, Loss: 0.12650
Epoch[2], Iter: 46, Loss: 0.38497
Epoch[2], Iter: 49, Loss: 0.04383
Epoch[2], Iter: 52, Loss: 0.00165
Epoch[2], Iter: 55, Loss: 0.36510
Epoch[2], Iter: 58, Loss: 0.07182
Epoch[2], Iter: 61, Loss: 0.20960
Epoch[2], Iter: 64, Loss: 0.24445
Epoch[2], Iter: 67, Loss: 0.00604
Epoch[2], Iter: 70, Loss: 0.70659
Training Results - Epoch: 2, Accuracy: 0.97509, Avg loss: 0.09561
Epoch[3], Iter: 73, Loss: 0.28930
Epoch[3], Iter: 76, L

***